In [22]:
!apt-get update
!apt-get install -y ffmpeg

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                 
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 103 not upgraded.


In [23]:
%pip install openai-whisper pandas datasets[audio]



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install torchcodec --index-url=https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
%pip install scipy librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from scipy.io import wavfile
from datasets import load_dataset
import torch
import os
import numpy as np

dataset = load_dataset("cubbk/audio_swedish_2_dataset_cleaned",split='train', streaming=True, data_dir="8sidor_audios_dataset") 

dataset_item = next(iter(dataset))

# Try different methods to get audio data from AudioDecoder
audio_decoder = dataset_item[".wav"] # type: ignore

audio_samples = audio_decoder.get_all_samples()

wav = audio_samples.data.numpy().squeeze()

audio_samples

# filepath = os.path.join("audio_segments", "aaaaaaaaaaa.wav")
# audio_segment_int16 = (wav * 32767).astype(np.int16)
# wavfile.write(filepath, 24000, audio_segment_int16)

In [ ]:
import whisper

model = whisper.load_model("turbo")

In [ ]:
import librosa

# Resample to 16kHz which is Whisper's expected sample rate
wav_resampled = librosa.resample(wav, orig_sr=audio_samples.sample_rate, target_sr=16000)
print(f"\nResampled audio:")
print(f"  New shape: {wav_resampled.shape}")
print(f"  New duration: {len(wav_resampled) / 16000:.2f} seconds")

# Transcribe the resampled audio
result = model.transcribe(
    wav_resampled,
    language="Swedish",
    verbose=True,
    temperature=0,
    condition_on_previous_text=False,
    # Try without any preprocessing that might affect timing
    no_speech_threshold=0.6,
    logprob_threshold=-1.0
)

segments = result["segments"]

# Check if timestamps make sense now
audio_duration = len(wav_resampled) / 16000
segments_cleaned = []

for i, segment in enumerate(segments):
    text = segment["text"].strip() # type: ignore
    if not text:
        continue
        
    start_time = segment["start"] # type: ignore
    end_time = segment["end"] # type: ignore
    
    # Debug: Print first few segments to see if timing looks reasonable
    if i < 5:
        print(f"Segment {i}: {start_time:.2f}s - {end_time:.2f}s | {text[:30]}...")
    
    # Only keep segments within audio duration
    if start_time < audio_duration: # type: ignore
        clipped_end = min(end_time, audio_duration)
        segments_cleaned.append({
            "start": start_time,
            "end": clipped_end,
            "text": text,
        })

print(f"\nKept {len(segments_cleaned)} segments out of {len(segments)}")


Original audio info:
  Shape: (11450410,)
  Sample rate: 24000
  Duration: 477.10 seconds
  Data type: float32
  Min/Max values: -1.000 / 1.000

Resampled audio:
  New shape: (7633607,)
  New duration: 477.10 seconds
[00:00.000 --> 00:03.960]  8sider.se fredagen den 13 mars.
[00:04.700 --> 00:05.580]  Sverige.
[00:06.380 --> 00:08.900]  Sverige stänger inte skolor.
[00:10.160 --> 00:13.360]  På torsdagen tog Sveriges regering ett nytt beslut.
[00:13.820 --> 00:15.900]  Det beror på viruset corona.
[00:16.720 --> 00:20.780]  I Norge och Danmark har skolor och förskolor stängt, men inte i Sverige.
[00:22.080 --> 00:28.040]  En orsak är att mammor och pappor som jobbar med till exempel vård behövs på sina jobb just nu.
[00:28.040 --> 00:33.200]  Det sa Anna Ekström på torsdagen. Hon är minister för skolor i Sverige.
[00:34.760 --> 00:39.520]  Regeringen har istället bestämt att skolan själva bestämmer om de ska vara öppna.
[00:40.100 --> 00:45.740]  Om smittan börjar spridas på en skola k

[{'start': 0.0, 'end': 3.96, 'text': '8sider.se fredagen den 13 mars.'},
 {'start': 4.7, 'end': 5.58, 'text': 'Sverige.'},
 {'start': 6.38, 'end': 8.9, 'text': 'Sverige stänger inte skolor.'},
 {'start': 10.16,
  'end': 13.36,
  'text': 'På torsdagen tog Sveriges regering ett nytt beslut.'},
 {'start': 13.82, 'end': 15.9, 'text': 'Det beror på viruset corona.'},
 {'start': 16.72,
  'end': 20.78,
  'text': 'I Norge och Danmark har skolor och förskolor stängt, men inte i Sverige.'},
 {'start': 22.080000000000002,
  'end': 28.04,
  'text': 'En orsak är att mammor och pappor som jobbar med till exempel vård behövs på sina jobb just nu.'},
 {'start': 28.04,
  'end': 33.2,
  'text': 'Det sa Anna Ekström på torsdagen. Hon är minister för skolor i Sverige.'},
 {'start': 34.76,
  'end': 39.519999999999996,
  'text': 'Regeringen har istället bestämt att skolan själva bestämmer om de ska vara öppna.'},
 {'start': 40.1,
  'end': 45.739999999999995,
  'text': 'Om smittan börjar spridas på en skola 

In [ ]:
import uuid

for i, segment in enumerate(segments_cleaned):
    uid = uuid.uuid4()
    start_time = segment["start"]
    end_time = segment["end"]
    text = segment["text"]
    
    start_sample = int(start_time * 24000)
    end_sample = int(end_time * 24000)

    segment_wav = wav[start_sample:end_sample]
    audio_segment_int16 = (segment_wav * 32767).astype(np.int16)
    
    filename = f"segment_{uid}.wav"
    filepath = os.path.join("audio_segments", filename)
    wavfile.write(filepath, 24000, audio_segment_int16)
    
    filename_text = f"segment_{uid}.txt"
    filepath_text = os.path.join("audio_segments", filename_text)
    with open(filepath_text, "w") as f:
        f.write(text)

FileNotFoundError: [Errno 2] No such file or directory: 'audio_segments/audio_segments/segment_81f4a3a1-28bf-4056-abfa-6cb81260a9d6.wav'